In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow.keras as K
import tensorflow as tf
import numpy as np

In [3]:
Train, Test = K.datasets.cifar10.load_data()
(x_train, y_train) = Train
(x_test, y_test) = Test

170500096/170498071 [==============================] - 11s 0us/step


In [4]:
def preprocess_data(X, Y):
    """
    * X is a numpy.ndarray of shape (m, 32, 32, 3) containing
        the CIFAR 10 data, where m is the number of data points
    * Y is a numpy.ndarray of shape (m,) containing the CIFAR 10
        labels for X
        Returns: X_p, Y_p
    * X_p is a numpy.ndarray containing the preprocessed X
    * Y_p is a numpy.ndarray containing the preprocessed Y
    """
    X_p = K.applications.densenet.preprocess_input(X)    
    Y_p = K.utils.to_categorical(y=Y, num_classes=10)
    return (X_p, Y_p)

In [5]:
xp_train, yp_train = preprocess_data(x_train, y_train)
xp_test, yp_test = preprocess_data(x_test, y_test)

In [6]:
lay_init = K.initializers.he_normal()
# pre_model  
entrada = K.Input(shape=(32, 32, 3))
resize = K.layers.Lambda(lambda image: tf.image.resize(image, (155, 155)))(entrada)
dense169 = K.applications.DenseNet169(include_top=False, weights="imagenet", input_tensor=resize)
# dense169.trainable = False
out_pre = dense169(resize)
 
vector = K.layers.Flatten()(out_pre)
drop1 = K.layers.Dropout(0.3)(vector)
norm_lay1 = K.layers.BatchNormalization()(drop1)
FC1 = K.layers.Dense(units=510, activation='relu', kernel_initializer=lay_init)(norm_lay1)
norm_lay2 = K.layers.BatchNormalization()(FC1)
out = K.layers.Dense(units=10, activation='softmax', kernel_initializer=lay_init)(norm_lay2)
 
model = K.models.Model(inputs=entrada, outputs=out)
   
learn_dec = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
early = K.callbacks.EarlyStopping(patience=5)
save = K.callbacks.ModelCheckpoint(filepath='cifar10.h5', save_best_only=True, monitor='val_loss', mode='min')

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x=xp_train, y=yp_train, batch_size=32, epochs=15, verbose=1, validation_data=(xp_test, yp_test), callbacks=[save, early, learn_dec])



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
51879936/51877672 [==============================] - 5s 0us/step
Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 1255s 25ms/sample - loss: 0.7569 - acc: 0.7532 - val_loss: 1.1165 - val_acc: 0.8094
Epoch 2/15
50000/50000 [==============================] - 1194s 24ms/sample - loss: 0.4532 - acc: 0.8474 - val_loss: 1.0631 - val_acc: 0.8134
Epoch 3/15
50000/50000 [==============================] - 1195s 24ms/sample - loss: 0.3927 - acc: 0.8683 - val_loss: 0.6551 - val_acc: 0.8370
Epoch 4/15
50000/50000 [==============================] - 1196s 24ms/sample - loss: 0.3140 - acc: 0.8925 - val_loss: 1.3636 - val_acc: 0.8252
Epoch 5/15
50000/50000 [==============================] - 1195s 24ms/sample - loss: 0.2528 - acc: 0.9125 - val_loss: 0.9293 - val_acc: 0.8748
Epoch 6/15
50000/50000 [==============================] - 1195s 24ms/sample - loss: 0.0944 - a

In [9]:
K.learning_phase = K.backend.learning_phase
K.is_sparse = K.backend.is_sparse
modelL = K.models.load_model('cifar10.h5')
modelL.evaluate(xp_test, yp_test, batch_size=128, verbose=1)


10000/10000 [==============================] - 49s 5ms/sample - loss: 0.3104 - acc: 0.9401


[0.3104340977191925, 0.9401]